In [206]:
import os
import pandas as pd
import numpy as np
import fastText
import pickle

In [69]:
egocom_loc = "/datasets/cgn/EGOCOM/"
word_vectors_loc = "/datasets/cgn/EGOCOM/crawl-300d-2M-subword.vec"

with open(word_vectors_loc, 'r') as rf:
    word_vectors = rf.read().splitlines()
num_word_vectors, dimension_word_vectors = (int(z) for z in word_vectors[0].split())
word_vectors = word_vectors[1:]
# Convert string to 
word_vectors = {w.split()[0]: np.array(w.split()[1:], dtype=np.float32) for w in word_vectors}

In [2]:
word_vector_model = fastText.load_model("/datasets/cgn/EGOCOM/crawl-300d-2M-subword.bin")

In [128]:
video_info = pd.read_csv(egocom_loc+"video_info.csv").drop('num_speakers', axis = 1)
df = pd.read_csv("/datasets/cgn/EGOCOM/ground_truth_transcriptions.csv")[
    ["conversation_id", "speaker_id", "startTime", "endTime", "word"]
]
df = pd.merge(df, video_info, on=["conversation_id", "speaker_id"], how = 'left')

In [129]:
# key_speaker_names = [
#     "person_{}_part".format(j[0][1]).join(j[0][0].split("part")) 
#      if 'part' in j[0][0] 
#      else j[0][0]+"__person_{}".format(j[0][1]) 
#      for i, j in enumerate(df.groupby(["conversation_id", "speaker_id"]))
# ]

In [ ]:
for history_length in [4, 5, 10, 30, 60]:
    print('history_length:', history_length, "\n")
    window_length = 1
    clip_ids, video_ids, features = [], [], []
    for video_id, sdf in df.groupby(["video_id"]):
        # HARD-CODED EXCEPTION: Ignore this 5 sec video
        if "day_1__con_5" in video_id and "part5" in video_id:
            continue

        start_time = 0
        end_time = history_length
        video_total_time = int(np.ceil(sdf["endTime"].max()))
        print(video_id, video_total_time)
        
        clips = list(range(0, video_total_time - history_length + 1, window_length))
        clip_ids += clips
        video_ids += [int(video_id[4:7])]*len(clips)

        for clip_id in clips:
            text = " ".join(sdf[(sdf['endTime'] >= start_time) & (sdf['endTime'] < end_time)]["word"])
            features.append(word_vector_model.get_sentence_vector(text))
            start_time += window_length
            end_time += window_length

    text_features = pd.DataFrame({
        "video_id" : video_ids,
        "clip_id" : clip_ids,
        "text_feature" : features,
    })
    
    # Expand columns
    expanded_text_feature = text_features.text_feature.apply(pd.Series)
    expanded_text_feature.rename(columns = lambda x : 'textfeat_' + str(x), inplace = True)
    text_features = pd.concat([text_features[:], expanded_text_feature[:]], axis=1)
    text_features = text_features.drop("text_feature", axis = 1)
    
    # Write out file
    wfn = egocom_loc + "text_features/egocom_text_window_1_sec_clip_duration_{}_sec.csv.gz".format(history_length)
    text_features.to_csv(wfn, index = False)

history_length: 4 

vid_001__day_1__con_1__person_1_part1 274
vid_002__day_1__con_1__person_1_part2 294
vid_003__day_1__con_1__person_1_part3 294
vid_004__day_1__con_1__person_1_part4 273
vid_005__day_1__con_1__person_1_part5 82
vid_006__day_1__con_1__person_2_part1 267
vid_007__day_1__con_1__person_2_part2 287
vid_008__day_1__con_1__person_2_part3 286
vid_009__day_1__con_1__person_2_part4 291
vid_010__day_1__con_1__person_2_part5 75
vid_011__day_1__con_1__person_3_part1 256
vid_012__day_1__con_1__person_3_part2 282
vid_013__day_1__con_1__person_3_part3 272
vid_014__day_1__con_1__person_3_part4 294
vid_015__day_1__con_1__person_3_part5 80
vid_016__day_1__con_2__person_1_part1 264
vid_017__day_1__con_2__person_1_part2 283
vid_018__day_1__con_2__person_1_part3 298
vid_019__day_1__con_2__person_1_part4 287
vid_020__day_1__con_2__person_1_part5 38
vid_021__day_1__con_2__person_2_part1 263
vid_022__day_1__con_2__person_2_part2 295
vid_023__day_1__con_2__person_2_part3 288
vid_024__day_1__co